## 4.4 自定义层

深度学习的一个魅力在于神经网络中各式各样的层，例如全链接层和后面章节中将要介绍的卷积层、池化层与循环层。

### 4.4.1 不含模型参数的自定义层

下面的CenteredLayer类通过继承Block类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了forward函数里。这个层里不含模型参数。

In [3]:
from mxnet import gluon, nd
from mxnet.gluon import nn

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def forward(self, x):
        return x - x.mean()

In [4]:
layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [5]:
net = nn.Sequential()
net.add(nn.Dense(128),
        CenteredLayer())

In [12]:
net.initialize()
y = net(nd.random_uniform(shape=(4, 8)))
y.mean().asscalar()

-7.421477e-10

### 4.4.2 含模型参数的自定义层

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

在自定义含模型参数的层时，我们可以利用Block类自带的ParameterDict类型的成员变量params。它是一个由字符串类型的参数名字映射到Prameter类型的模型参数的字典。

In [13]:
params = gluon.ParameterDict()
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [14]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super().__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))
        
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

实例化MyDense类并访问它的模型参数

In [15]:
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

直接使用自定义层做前向计算

In [16]:
dense.initialize()
dense(nd.random_uniform(shape=(2, 5)))


[[0.         0.04648931 0.09400678]
 [0.         0.         0.02732784]]
<NDArray 2x3 @cpu(0)>

使用自定义层构造模型

In [19]:
net = nn.Sequential()
net.add(MyDense(units=8, in_units=64),
        MyDense(units=1, in_units=8))
net.initialize()
net(nd.random_uniform(shape=(2, 64)))


[[0.]
 [0.]]
<NDArray 2x1 @cpu(0)>

### 小结

- 可以通过Block类自定义神经网络中的层，从而可以被重复调用。

### 练习

- 自定义一个层，使用它做一次前向计算。

In [20]:
# 定义一个层
class MyFavouriteDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super().__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))
        
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.sigmoid(linear)
    

In [21]:
# 实例化MyFavouriteDense类并访问它的模型参数
dense = MyFavouriteDense(units=9, in_units=12)
dense.params

myfavouritedense0_ (
  Parameter myfavouritedense0_weight (shape=(12, 9), dtype=<class 'numpy.float32'>)
  Parameter myfavouritedense0_bias (shape=(9,), dtype=<class 'numpy.float32'>)
)

In [22]:
# 使用自定义层做前向计算
dense.initialize()
dense(nd.random_uniform(shape=(50, 12)))


[[0.49905163 0.49295434 0.5168587  0.4825667  0.5245432  0.4909766
  0.5034553  0.48245028 0.45977002]
 [0.49888033 0.49145684 0.52476823 0.49484724 0.53001577 0.4946552
  0.50558215 0.4857613  0.45687237]
 [0.5083423  0.46598774 0.5103539  0.4894063  0.544457   0.4700225
  0.53259134 0.5102469  0.49325272]
 [0.4959993  0.49639025 0.5172226  0.4860539  0.52686733 0.48333624
  0.50136    0.49174598 0.4742024 ]
 [0.501645   0.4874378  0.5178983  0.48653585 0.547457   0.48724288
  0.5272622  0.5117731  0.4895995 ]
 [0.51400006 0.46848547 0.5123003  0.49393123 0.56081426 0.47711363
  0.5343128  0.4905582  0.4746947 ]
 [0.52215135 0.46523327 0.51617795 0.5023349  0.5504886  0.47045365
  0.5270774  0.5174171  0.48272058]
 [0.51155794 0.49104097 0.5438664  0.51329374 0.54477245 0.49041986
  0.51305145 0.5159679  0.4611001 ]
 [0.50187725 0.49816912 0.509491   0.49487978 0.5290062  0.47594476
  0.5120902  0.51532465 0.47845086]
 [0.50262594 0.491421   0.5157757  0.48444003 0.5297511  0.4962814